In [1]:
import json
import urllib.parse
import urllib.request
import base64
from datetime import datetime
import openpyxl  # Importujemy bibliotekę openpyxl do odczytu plików Excel
import pandas as pd  # Używamy pandas do pracy z DataFrame
from urllib.error import HTTPError  # Importujemy wyjątek HTTPError

# Pobieramy dane wejściowe (URL, login, hasło) z argumentów wiersza poleceń
service_url = 'https://shapi.imgw.pl/aggregation/read-many'  # Zmień na swój URL
user_login = 'alejur'                  # <------------------------------------------------------ Zmień na swój login
user_password = 'qwer1234'             # <------------------------------------------------------- Zmień na swoje hasło

# Przygotowanie nagłówka autoryzacji
auth_value = base64.b64encode(f'{user_login}:{user_password}'.encode()).decode('utf-8')
headers = {
    'Authorization': f'Basic {auth_value}',
    'Content-Type': 'application/json'  # Dodanie nagłówka Content-Type dla JSON
}

# Przygotowanie parametrów zapytania
values = {
    'from': '20250501060000',    # <-------------------------------------------- zmiana dat
    'to':   '20250601235900',    # <-------------------------------------------- zmiana dat
}

# Funkcja do wczytania danych stacji z pliku Excel, wybierając konkretny arkusz
def get_station_data_from_xlsx(file_path, sheet_name):
    stations = []
    wb = openpyxl.load_workbook(file_path)
    
    # Wybór arkusza po nazwie
    sheet = wb[sheet_name]  
    
    for row in sheet.iter_rows(min_row=1, max_col=9, values_only=True):  # Zaczynamy od drugiego wiersza, pomijamy nagłówki
        station_id = row[3]  # Kolumna D
        operation_id = row[1]  # Kolumna B
        
        # Dodajemy dane stacji do listy, tylko jeśli obie wartości są obecne
        if station_id and operation_id:
            stations.append({
                "dre-name": station_id,
                "operation-id": operation_id
            })
    
    return stations

# Wczytujemy dane stacji z pliku Excel
stations = get_station_data_from_xlsx('lista_stacji_do_SH_API.xlsx', 'OPAD_MIES')  # Ścieżka do pliku Excel z danymi stacji

# Przygotowanie pełnego URL i żądanie HTTP
url_values = urllib.parse.urlencode(values)
url = f'{service_url}?{url_values}'

# Lista stacji, które spowodowały błąd
failed_stations = []

# Lista, do której będziemy dodawać dane przed utworzeniem DataFrame
data = []

# Iterujemy po stacjach
for station in stations:
    # Przygotowanie danych stacji do wysłania
    json_to_send = [station]  # Wysyłamy tylko jedną stację w zapytaniu

    req = urllib.request.Request(url, headers=headers, method='POST')
    data_request = json.dumps(json_to_send).encode('utf-8')
    req.data = data_request  # Dodanie danych JSON do żądania

    try:
        # Wysłanie zapytania i odebranie odpowiedzi
        with urllib.request.urlopen(req) as response:
            content = response.read()

        records = json.loads(content)

        # Przetwarzanie odpowiedzi
        for record in records:
            if 'dre-name' in record and record["dre-name"]:
                # Przekształcamy datę z formatu "YYYYMMDDhhmmss" na datetime
                date = datetime.strptime(str(record["date"]), '%Y%m%d%H%M%S')
                
                # Dodajemy dane do listy przed utworzeniem DataFrame
                data.append([record["dre-name"], date.strftime('%Y-%m-%d %H:%M:%S'), record["value"]])

                # Opcjonalnie, wypisz wynik na konsolę
                print(f'{record["dre-name"]} {date} = {record["value"]}')
    except HTTPError as e:
        # Jeśli napotkamy błąd 403, dodajemy stację do listy failed_stations
        if e.code == 403:
            print(f'Błąd 403: Dostęp zabroniony dla stacji {station["dre-name"]}')
            failed_stations.append(station["dre-name"])
        else:
            # Inne błędy HTTP
            print(f'Błąd HTTP {e.code} dla stacji {station["dre-name"]}')

# Jeśli były stacje, które spowodowały błąd 403, zapisujemy je do pliku
if failed_stations:
    with open('failed_stations.txt', 'w', encoding='utf-8') as failed_file:
        for station in failed_stations:
            failed_file.write(str(station) + '\n')

# Tworzymy DataFrame z danych
df = pd.DataFrame(data, columns=['dre-name', 'date', 'value'])

# Opcjonalnie, wypisz dane w DataFrame
print(df)


# Opcjonalnie, wypisz informacje o stacjach, które spowodowały błąd
if failed_stations:
    print(f"Stacje, które spowodowały błąd 403: {', '.join(map(str, failed_stations))}")

# Pobierz dzisiejszą datę w formacie RRRRMMDD
today_str = datetime.today().strftime('%Y%m%d')
# Składanie ścieżki z datą
file_path = f'B:/CHOK/BMPIS/CMPIS-K/2_wymiana_Danych/__SUSZA/BIULETYN/01_BIULETYN_OLA_RADEK/SPI_30_04/0_Dane_do_SPI/opad_mies_{today_str}.csv'
# Zapis do pliku
df.to_csv(file_path, index=False, sep=';')

354220195 2025-06-01 06:00:00 = 45.6
354220195 2025-05-01 06:00:00 = 10.2
354210185 2025-06-01 06:00:00 = 51.1
354210185 2025-05-01 06:00:00 = 19.0
354190160 2025-06-01 06:00:00 = 81.8
354190160 2025-05-01 06:00:00 = 16.2
354180155 2025-06-01 06:00:00 = 43.2
354180155 2025-05-01 06:00:00 = 12.9
354180135 2025-06-01 06:00:00 = 35.7
354180135 2025-05-01 06:00:00 = 22.7
354170120 2025-06-01 06:00:00 = 41.2
354170120 2025-05-01 06:00:00 = 10.4
354160115 2025-06-01 06:00:00 = 25.21
354160115 2025-05-01 06:00:00 = 7.41
354160105 2025-06-01 06:00:00 = 46.8
354160105 2025-05-01 06:00:00 = 6.6
354150100 2025-06-01 06:00:00 = 27.2
354150100 2025-05-01 06:00:00 = 2.51
353230295 2025-06-01 06:00:00 = 47.5
353230295 2025-05-01 06:00:00 = 9.7
353210285 2025-06-01 06:00:00 = 78.7
353210285 2025-05-01 06:00:00 = 21.1
353210280 2025-05-01 06:00:00 = 24.1
353200272 2025-06-01 06:00:00 = 84.9
353200272 2025-05-01 06:00:00 = 17.2
353200270 2025-06-01 06:00:00 = 49.4
353200270 2025-05-01 06:00:00 = 7.6
353

OSError: Cannot save file into a non-existent directory: 'B:\CHOK\BMPIS\CMPIS-K\2_wymiana_Danych\SUSZA\BIULETYN\01_BIULETYN_OLA_RADEK\SPI_30_04\0_Dane_do_SPI'